# Facebook Freunde Scrapen
- Eine super Datenquelle um Netzwerke zu verstehen sind die eigenen Freunde. 
- Diese lassen sich von Facebook scrapen, da Facebook natürlich keinen Export erlaubt - d'oh. 

In [23]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import json
import csv
import re

In [41]:
s = webdriver.Chrome('/usr/local/bin/chromedriver')

https://selenium-python.readthedocs.io/api.html#selenium.webdriver.remote.webdriver.WebDriver.find_element_by_link_text- 

In [32]:
USERNAME = "priskawa@me.com"
PASSWORD = "Montpellier34+"
MY_SLUG = "priska.wallimann"

In [33]:
def login(s):
    try:
        print("Logging into FB")
        s.get("http://facebook.com/login")
        s.find_element_by_id("email").send_keys(USERNAME)
        r = s.find_element_by_id("pass")
        r.send_keys(PASSWORD)
        r.send_keys(Keys.RETURN)
    except:
        print("Probably already logged in...")

In [34]:
def scrape_friends(s):
    login(s)
    #Scrape my friends
    s.get("https://www.facebook.com/%s/friends" % MY_SLUG)
    for i in range(30):
        print("scrolling...and...waiting..%s" % i)
        s.execute_script('window.scroll(0,100000);')
        time.sleep(1)
    # get html
    elem = s.find_element_by_xpath("//*")
    source_code = elem.get_attribute("outerHTML")
    soup = BeautifulSoup(source_code, 'lxml')
    friends = soup.select(".fcb a")
    print("Scraping {%s} friends" % len(friends))
    with open('friends.csv', mode='w') as myfile:
        mycsv = csv.writer(myfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        mycsv.writerow(["Friend ID", "Friend Name"])
        for friend in friends:
          try:
            friend_name = friend.text.encode("ascii","ignore")
            friend_id = json.loads(friend["data-gt"])["engagement"]["eng_tid"].encode("ascii","ignore")
            mycsv.writerow([friend_id, friend_name])
          except:
            print("Problems obtaining %s" % friend)

In [35]:
scrape_friends(s)

Logging into FB
scrolling...and...waiting..0
scrolling...and...waiting..1
scrolling...and...waiting..2
scrolling...and...waiting..3
scrolling...and...waiting..4
scrolling...and...waiting..5
scrolling...and...waiting..6
scrolling...and...waiting..7
scrolling...and...waiting..8
scrolling...and...waiting..9
scrolling...and...waiting..10
scrolling...and...waiting..11
scrolling...and...waiting..12
scrolling...and...waiting..13
scrolling...and...waiting..14
scrolling...and...waiting..15
scrolling...and...waiting..16
scrolling...and...waiting..17
scrolling...and...waiting..18
scrolling...and...waiting..19
scrolling...and...waiting..20
scrolling...and...waiting..21
scrolling...and...waiting..22
scrolling...and...waiting..23
scrolling...and...waiting..24
scrolling...and...waiting..25
scrolling...and...waiting..26
scrolling...and...waiting..27
scrolling...and...waiting..28
scrolling...and...waiting..29
Scraping {285} friends
Problems obtaining <a ajaxify="/ajax/friends/inactive/dialog?id=1000022

In [42]:
def scrape_mutual_friends(s):
    login(s)
    edgelist = []
    friendslist = []
    
    #Get FriendsIDs
    with open('friends.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for i, row in enumerate(csv_reader):
            if i != 0:
                friendslist.append([row["Friend ID"],row["Friend Name"]])
                
    #Get my Id
    s.get ("http://facebook.com/%s" % MY_SLUG)
    for i in range(30):
        print("scrolling...and...waiting..%s" % i)
        s.execute_script('window.scroll(0,100000);')
        time.sleep(1)
    elem = s.find_element_by_xpath("//*")
    source_code = elem.get_attribute("outerHTML")
    soup = BeautifulSoup(source_code, 'lxml')
    my_id = re.search(r"profile_id=(\d*)", source_code).groups()[0]
    print("My ID: %s" % my_id)

    for friends in friendslist:
        print("Visiting friend: %s" % friends[1])
        s.get("https://www.facebook.com/browse/mutual_friends/?uid=%s&node=%s" % (my_id,friends[0][2:-1]))
        elem = s.find_element_by_xpath("//*")
        source_code = elem.get_attribute("outerHTML")
        soup = BeautifulSoup(source_code, 'lxml')
        mutual_friends = soup.select(".fcb a")
        print("Found %s mutual friends" % len(mutual_friends))
        for friend in mutual_friends:
            edgelist.append([friends[0],json.loads(friend["data-gt"])["engagement"]["eng_tid"].encode("ascii","ignore")])

    with open('network.csv', mode='w') as myfile:
        mycsv = csv.writer(myfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        mycsv.writerow(["From","To"])
        for edge in edgelist:
            mycsv.writerow([edge[0],edge[1]])

In [43]:
scrape_mutual_friends(s)

Logging into FB
scrolling...and...waiting..0
scrolling...and...waiting..1
scrolling...and...waiting..2
scrolling...and...waiting..3
scrolling...and...waiting..4
scrolling...and...waiting..5
scrolling...and...waiting..6
scrolling...and...waiting..7
scrolling...and...waiting..8
scrolling...and...waiting..9
scrolling...and...waiting..10
scrolling...and...waiting..11
scrolling...and...waiting..12
scrolling...and...waiting..13
scrolling...and...waiting..14
scrolling...and...waiting..15
scrolling...and...waiting..16
scrolling...and...waiting..17
scrolling...and...waiting..18
scrolling...and...waiting..19
scrolling...and...waiting..20
scrolling...and...waiting..21
scrolling...and...waiting..22
scrolling...and...waiting..23
scrolling...and...waiting..24
scrolling...and...waiting..25
scrolling...and...waiting..26
scrolling...and...waiting..27
scrolling...and...waiting..28
scrolling...and...waiting..29
My ID: 1446190064
Visiting friend: b'Tamara Keta'
Found 3 mutual friends
Visiting friend: b'Li

Found 1 mutual friends
Visiting friend: b'Levke Nissen'
Found 2 mutual friends
Visiting friend: b'Rolf Cavalli'
Found 28 mutual friends
Visiting friend: b'Dom Hug'
Found 28 mutual friends
Visiting friend: b'Andr Maerz'
Found 30 mutual friends
Visiting friend: b'Simone Hufschmid'
Found 3 mutual friends
Visiting friend: b'Jasmin Kirchhofer'
Found 1 mutual friends
Visiting friend: b'Janine Gygax'
Found 2 mutual friends
Visiting friend: b'Monika Molnarova'
Found 1 mutual friends
Visiting friend: b'Filip Matuinsk'
Found 2 mutual friends
Visiting friend: b'Bolliger Daniela'
Found 9 mutual friends
Visiting friend: b'Kathrin Hefel'
Found 20 mutual friends
Visiting friend: b'Andrea Cristina Trueb'
Found 30 mutual friends
Visiting friend: b'Conny Spaltenstein'
Found 8 mutual friends
Visiting friend: b'Christoph Sprgin'
Found 20 mutual friends
Visiting friend: b'Gabriela Gehringer'
Found 6 mutual friends
Visiting friend: b'Marcel Zulauf'
Found 30 mutual friends
Visiting friend: b'Markus Helbling'

Found 2 mutual friends
Visiting friend: b'Marco Schwarz'
Found 2 mutual friends
Visiting friend: b'Marta Doblas'
Found 10 mutual friends
Visiting friend: b'Anne Dentan'
Found 7 mutual friends
Visiting friend: b'Marcel Engelberger'
Found 8 mutual friends
Visiting friend: b'Toto Marti'
Found 30 mutual friends
Visiting friend: b'Wolfgang Buechner'
Found 19 mutual friends
Visiting friend: b'Michael Huber'
Found 29 mutual friends
Visiting friend: b'Adolfo Arranz'
Found 15 mutual friends
Visiting friend: b'Peter Wolf'
Found 30 mutual friends


In [44]:
import networkx as nx

def create_user_network():
    friendslist = []
    edgelist = []
    
    with open('friends.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for i, row in enumerate(csv_reader):
            if i != 0:
                friendslist.append([row["Friend ID"],row["Friend Name"]])        

    with open('network.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for i, row in enumerate(csv_reader):
            if i != 0:
                edgelist.append([row["From"],row["To"]])
    
    G  = nx.Graph(name="Friends")
                            
    for friend in friendslist:
        G.add_node(friend[0],label=friend[1], size=1)

    for edge in edgelist:
        G.add_edge(edge[0],edge[1],weight=1)
   
    nx.write_gexf(G,"Friends.gexf")

In [45]:
create_user_network()